# Lots de tâches
Le calcul haute-performance consiste non seulement au calcul parallèle par tâche (***parallélisme des tâches***),
mais aussi au calcul de données en parallèle dans plusieurs tâches et/ou processus en simultané (***parallélisme de données***).
Ce chapitre vous donnera les outils nécessaires pour gérer un grand nombre de tâches
lorsque le projet de recherche requiert plusieurs centaines de résultats.

## GNU Parallel
La commande `parallel` de [GNU Parallel](https://docs.computecanada.ca/wiki/GNU_Parallel/fr) permet d'utiliser pleinement les ressources locales d'un noeud de calcul, et ce, en gérant l'exécution d'une **longue liste de tâches de *petite* taille**.
C'est un peu comme l'ordonnanceur Slurm, mais à plus petite échelle et en gérant des processus au lieu de scripts de tâche.
* [Documentation officielle](https://www.gnu.org/software/parallel/parallel.html)
* [Tutoriel](https://www.gnu.org/software/parallel/parallel_tutorial.html)

### Pourquoi pas Slurm?
OK, mais pourquoi ne pas tout simplement soumettre **des centaines de tâches à Slurm**?
* À tout moment, Slurm **limite chaque usager à 1000 tâches** au total dans `squeue` (*pending* + *running*)
* Certains calculs sont tellement **courts (< 5 minutes)** que le démarrage et la fin de la tâche Slurm compteraient pour un pourcentage significatif du temps réel utilisé
  * On veut donc réduire les pertes de temps
* GNU Parallel nous **évite d'utiliser une boucle** soumettant des centaines de scripts similaires, ce qui, dans bien des cas, facilite l'exécution de centaines de tâches de calcul semblables
* Enfin, si toutes vos tâches envoient des **notifications par courriel**, imaginez recevoir ces centaines de courriels!

Les autres avantages à considérer :
* Par défaut, le nombre de **processeurs disponibles limite** le nombre de tâches de calcul exécutées en simultané
  * Dans le cas de calculs parallèles, c'est possible de limiter davantage le nombre de processus
* GNU Parallel peut [reprendre la séquence des tâches de calcul](https://docs.computecanada.ca/wiki/GNU_Parallel/fr#Suivi_des_commandes_ex.C3.A9cut.C3.A9es_ou_des_commandes_ayant_.C3.A9chou.C3.A9.3B_fonctionnalit.C3.A9s_de_red.C3.A9marrage) en cas de fin hâtive de la tâche Slurm

### Syntaxe de la commande GNU Parallel
Les éléments de base de la commande `parallel` : options, gabarit de commande, liste de valeurs. Voir :
```Bash
man parallel
```

### Cas d'utilisation
#### Séquence de paramètre
Le paramètre changeant est donné via une paire d'`{}` :
```Bash
parallel echo fichier{}.txt ::: 1 2 3 4
parallel --citation  # Il faut citer les développeurs
```

Utilisation d'une séquence générée par la commande `seq` :
```Bash
seq 1 20
seq -w 1 20
echo $(seq -w 1 20)
parallel echo fichier{}.txt ::: $(seq -w 1 20)
```

#### Combinaisons de paramètres
Les paramètres changeants peuvent être donnés via une paire d'`{}`, ou plus précisément par des paire d'`{}` numérotées telles que `{1}`, `{2}`, etc. :
```Bash
parallel echo fichier{}.txt ::: $(seq -w 1 10) ::: a b
parallel echo fichier{1}{2}.txt ::: $(seq -w 1 10) ::: a b
```

Dans le cas où on retrouve les **combinaisons de paramètres dans un fichier texte** :
```Bash
# parallel echo {} ::: 3 5 7 ::: 4 6 8 > param.txt
bash scripts/prll-creer-param.sh
```

Par la suite, dans le script de tâche Slurm :
```Bash
# parallel -C ' ' echo '{1}*{2} | bc > prod_{1}x{2}' :::: param.txt
sbatch scripts/prll-exec-param.sh
```

Si on préfère valider la liste des commandes générées avant de soumettre
une tâche Slurm, on peut générer une **liste de commandes dans un fichier texte** :
```Bash
# parallel -C ' ' echo 'echo {1}"*"{2} "|" bc ">" prod_{1}x{2}' :::: param.txt > commandes.txt
bash scripts/prll-creer-cmd.sh
```

Et dans la tâche Slurm :
```Bash
# parallel < commandes.txt
sbatch scripts/prll-exec-cmd.sh
```

#### Limiter le nombre de tâches parallèles
Le paramètre `--jobs` permet de forcer une limite sur le nombre de processus lancés à la fois. Par exemple, 8 exécutions avec 2 processus en simultané :
```Bash
parallel --jobs 2 'echo {} && sleep 3' ::: $(seq -w 1 8)
```

Si jamais un noeud n'est pas suffisant pour répondre au besoin de centaines de tâches parallèles, GNU Parallel peut distribuer des tâches sur plusieurs noeuds
```Bash
# parallel \
#    --jobs $SLURM_NTASKS_PER_NODE \
#    --sshloginfile noeuds.txt \
#   'echo -n {}: && hostname' ::: $(seq 1 $SLURM_NTASKS)
sbatch scripts/prll-multi-noeud.sh
```

### Exercice - Aligner des séquences d'ADN
Dans le dossier `donnees/`, vous devriez avoir plusieurs fichiers Fasta (`*.fa`)
qui ont été créés au premier chapitre via le script de tâche
`scripts/blastn-gen-seq.sh` et le Makefile `donnees/makefile`.
**Si ce n'est pas le cas**, faites :
```Bash
sbatch scripts/blastn-gen-seq.sh
```

On devrait y retrouver :
* pour chaque espèce fictive A et B, un fichier de séquences d'ADN "connues"
  * Ces fichiers sont convertis en bases de données Blast
* pour 6 espèces "inconnues" M à R, des séquences d'ADN à aligner sur les séquences "connues" des espèces A et B

On cherche donc à calculer l'alignement de toutes les combinaisons
`{A,B}x[M,R]`, ce qui donne 12 combinaisons.
Le script de tâche suivant utilise GNU Parallel pour calculer les
différents cas avec 4 processeurs :
```Bash
cat scripts/blastn-parallel.sh
sbatch scripts/blastn-parallel.sh
```

Pour surveiller l'état de la tâche :
```Bash
squeue -u $USER
ls donnees/res_prll/
```
À la fin de la tâche, on peut analyser les ressources utilisées :
```
seff <jobid>
```

## Vecteurs de tâches
Alors que GNU Parallel s'utilise bien avec un lot de calculs séquentiels
ou parallèles de petite taille (16 processeurs ou moins),
**GNU Parallel n'est pas** vraiment l'outil **approprié pour**
un lot de **longs calculs parallèles de plus grande taille**
(plus de 16 processeurs par calcul) :
* on veut éviter les longues tâches qui dépassent trois (3) jours et
* on veut réduire le risque de subir une défaillance matérielle.

Dans le cas où un même programme parallèle doit être exécuté avec
différentes combinaisons de paramètres, il y a moyen de soumettre un seul
[vecteur de tâches](https://docs.computecanada.ca/wiki/Job_arrays/fr)
et de coder le script de tâche de telle sorte que les paramètres
seront déterminés **en fonction d'un indice unique** du vecteur de tâches.

**Pour soumettre un vecteur de tâches** à l'ordonnanceur Slurm, à la ligne de commande `sbatch` ou dans l'entête `#SBATCH` du script de tâche, **on doit ajouter l'option** `--array=<indices>`.
Voir [ici quelques exemples](https://docs.computecanada.ca/wiki/Job_arrays/fr).

L'identifiant d'une tâche Slurm dans un vecteur de tâches contient :
* L'identifiant du vecteur de tâches
* Caractère de soulignement (`_`)
* L'indice unique associé à la tâche

**Par exemple :** `25249551_15`

Dans le script de tâche, la **variable d'environnement**
`$SLURM_ARRAY_TASK_ID` peut être utilisée pour retrouver la valeur
actuelle de l'indice unique associé à la tâche en cours d'exécution.
Il s'agit d'un **entier parmis** les `<indices>`.
```Bash
export SLURM_ARRAY_TASK_ID=71  # Pour émuler la variable
```

La variable `$SLURM_ARRAY_TASK_ID` peut être utilisée de différentes façons :
```Bash
echo nom_fichier.$SLURM_ARRAY_TASK_ID
echo nom_dossier-$SLURM_ARRAY_TASK_ID

PARAM=$SLURM_ARRAY_TASK_ID
echo $PARAM

PARAM_R=$((PARAM / 12))  # Division entière
PARAM_C=$((PARAM % 12))  # Modulo (reste de division)
echo $PARAM_R $PARAM_C

echo $(sed -n "${PARAM_R}p" param.txt)
```

### Exercice - Vecteur de tâches
Pour lancer le vecteur de tâches :
```Bash
cat scripts/blastn-array.sh
sbatch scripts/blastn-array.sh

squeue -u $USER
```

Après l'exécution des quatre tâches, on peut inspecter les résultats :
```Bash
ls slurm-*_*.out
ls -l donnees/res_array/
```

## Le *Greedy Launcher Of Small Tasks*, ou GLOST
Le *Greedy Launcher Of Small Tasks*, ou [GLOST](https://docs.computecanada.ca/wiki/GLOST/fr), fonctionne un peu **comme GNU Parallel, mais avec quelques différences** :
* Fonctionne qu'avec des **tâches séquentielles (un processeur par tâche)**
* L'ordonnancement des processus se fait via **[MPI](https://docs.computecanada.ca/wiki/MPI/fr)** avec une architecture de type **gestionnaire-travailleur**
  * Le gestionnaire envoie à chaque travailleur une tâche séquentielle à exécuter
  * Le travailleur signale au gestionnaire quand le travail est complété
  * Le gestionnaire informe les travailleurs lorsqu'il n'y a plus de tâche (pour quitter)
* On doit définir **une ligne de commande par tâche** :
  * Toutes les lignes de commande sont **[listées dans un même fichier](https://docs.computecanada.ca/wiki/GLOST/fr#Liste_de_t.C3.A2ches_situ.C3.A9es_dans_le_m.C3.AAme_r.C3.A9pertoire)**
  * Les sous-commandes peuvent être séparées par l'opérateur `&&`. Par exemple :

```Bash
echo -n 'Bonjour ' && echo le monde
```

### Exercice - Liste de tâches Blastn avec GLOST
Pour générer la liste de tâches dans le fichier `$SCRATCH/cas-blastn.txt` :
```Bash
cat scripts/blastn-cas.sh
bash scripts/blastn-cas.sh

cat $SCRATCH/cas-blastn.txt
```

Pour lancer une tâche de 4 processeurs utilisant GLOST pour tester les $3*2*2$ combinaisons de paramètres :
```Bash
cat scripts/blastn-glost.sh
sbatch scripts/blastn-glost.sh

squeue -u $USER
```

On peut aller voir les processus en exécution :
```Bash
ssh <nom_noeud>
top -u $USER  # q pour quitter
exit
```

Les résultats seront dans le dossier `$SCRATCH/res_glost`
```Bash
ls -l $SCRATCH/res_glost
```